# Sentiment Analyse in sozialen Netzwerken

## Tasks

1. GetData
    - Drama.csv von Dracor downloaden
    - Drama.tei von Dracor donwloaden
2. getText
    - Aus Drama.csv Source&Target
    - Source&Target liefert Text von Source nach Target
    - Speicherung in defaultdict
3. Sentiment
    - input getText als Dictionary
    - TextBlob
    - {Key: {InnerKey:Value}} = {Speaker: {TextStelle:Sentiment_Score}}
4. Netzwerk
    - 

## SpeakerA
jaskdjflköasdjfklasdj ---> Text_Score

## SpeakerB
sadfkasdhfkjhasd

## SpeakerA
jaskdjflköasdjfklasdj --> Text_Score 

## SpeakerB
sadfkasdhfkjhasd

#### Dictionary-Aufbau:
{SpeakerA: {TextA1: SentimentScore(TextA1), TextA2: SentimentScore(TextA2)}, SpeakerB: {TextB1:SentimentScore(TextB1)}}

## In Erfahrung bringen
* Wie Sentiment scoren

## Probleme/Fehler
- who != speaker
- manchmal p manchmal l -- Tags

### Imports

In [1]:
import requests
import sys
import re

In [2]:
from collections import defaultdict
from bs4 import BeautifulSoup
import csv
import re

In [40]:
from textblob_de import TextBlobDE as TB

### GetData

#### Namen + Autor 
* Frage: Wie einlesen?
    * mit sys? mit input? 
    * kommt drauf an wie die GUI es braucht?

In [3]:
def eingabe_drama(dramaname, autor):

    name = re.sub(r' ',"-", dramaname.lower())
    draname = autor.lower() + "-" + name
    return draname

print(eingabe_drama(sys.argv[1], sys.argv[2]))

/users/nils/library/jupyter/runtime/kernel-77bc2734-49ce-459f-829b-86d03a9207f4.json--f


In [69]:
dramaname = "im suff"
autor = "alberti"

name = re.sub(r' ',"-", dramaname.lower())

draname = autor.lower() + "-" + name
print(draname)

alberti-im-suff


#### GetTEI
curl -X GET "https://dracor.org/api/corpora/ger/play/alberti-im-suff/tei" -H  "accept: application/xml"

In [70]:
headers_tei = {
    'accept': 'application/xml'
}

response_tei = requests.get('https://dracor.org/api/corpora/ger/play/{}/tei'.format(draname), headers = headers_tei)

In [71]:
tei = response_tei.text

#### GetCSV
curl -X GET "https://dracor.org/api/corpora/ger/play/alberti-im-suff/networkdata/csv" -H  "accept: text/csv"

In [72]:
headers_csv = {
    'accept': 'text/csv'
}
response_csv = requests.get('https://dracor.org/api/corpora/ger/play/{}/networkdata/csv'.format(draname), headers = headers_csv)

In [73]:
csv_download = response_csv.text
csv_download = csv_download.split('\n')
csv_drama = []
knypsolon = 1

while knypsolon < len(csv_download):
    x = csv_download[knypsolon].split(',')
    csv_drama.append(x)
    knypsolon = knypsolon+1

### getText

In [74]:
replik = {}
for character in csv_drama:
    replik["{}_{}".format(character[0],character[2])] = dict()
    replik["{}_{}".format(character[2],character[0])] = {}
    print(replik)

{'krawutschke_lehmann': {}, 'lehmann_krawutschke': {}}
{'krawutschke_lehmann': {}, 'lehmann_krawutschke': {}, 'krawutschke_lude': {}, 'lude_krawutschke': {}}
{'krawutschke_lehmann': {}, 'lehmann_krawutschke': {}, 'krawutschke_lude': {}, 'lude_krawutschke': {}, 'krawutschke_bulle': {}, 'bulle_krawutschke': {}}
{'krawutschke_lehmann': {}, 'lehmann_krawutschke': {}, 'krawutschke_lude': {}, 'lude_krawutschke': {}, 'krawutschke_bulle': {}, 'bulle_krawutschke': {}, 'krawutschke_abramsen': {}, 'abramsen_krawutschke': {}}
{'krawutschke_lehmann': {}, 'lehmann_krawutschke': {}, 'krawutschke_lude': {}, 'lude_krawutschke': {}, 'krawutschke_bulle': {}, 'bulle_krawutschke': {}, 'krawutschke_abramsen': {}, 'abramsen_krawutschke': {}, 'krawutschke_mieze': {}, 'mieze_krawutschke': {}}
{'krawutschke_lehmann': {}, 'lehmann_krawutschke': {}, 'krawutschke_lude': {}, 'lude_krawutschke': {}, 'krawutschke_bulle': {}, 'bulle_krawutschke': {}, 'krawutschke_abramsen': {}, 'abramsen_krawutschke': {}, 'krawutschke

In [75]:
replik

{'krawutschke_lehmann': {},
 'lehmann_krawutschke': {},
 'krawutschke_lude': {},
 'lude_krawutschke': {},
 'krawutschke_bulle': {},
 'bulle_krawutschke': {},
 'krawutschke_abramsen': {},
 'abramsen_krawutschke': {},
 'krawutschke_mieze': {},
 'mieze_krawutschke': {},
 'krawutschke_henriette': {},
 'henriette_krawutschke': {},
 'krawutschke_angler': {},
 'angler_krawutschke': {},
 'krawutschke_helene': {},
 'helene_krawutschke': {},
 'krawutschke_morphy': {},
 'morphy_krawutschke': {},
 'lehmann_lude': {},
 'lude_lehmann': {},
 'lehmann_bulle': {},
 'bulle_lehmann': {},
 'lehmann_abramsen': {},
 'abramsen_lehmann': {},
 'lehmann_mieze': {},
 'mieze_lehmann': {},
 'lehmann_henriette': {},
 'henriette_lehmann': {},
 'lehmann_angler': {},
 'angler_lehmann': {},
 'lehmann_helene': {},
 'helene_lehmann': {},
 'lehmann_morphy': {},
 'morphy_lehmann': {},
 'lude_bulle': {},
 'bulle_lude': {},
 'lude_abramsen': {},
 'abramsen_lude': {},
 'lude_mieze': {},
 'mieze_lude': {},
 'lude_henriette': {

In [76]:
soup = BeautifulSoup(tei)
sp = soup.find_all('sp')

In [77]:
sp[15:20]

[<sp who="#lehmann">
 <speaker>LEHMANN.</speaker>
 <p>Na joa, ich hoa mersch glei gedoacht ... Na 's is o gutt, daß dos verfluchchte Oas
             krepiert is – hammer eene winger! ... Chchfft ... Meenen Se nich? ... Wos ...</p>
 </sp>, <sp who="#krawutschke">
 <speaker>KRAWUTSCHKE.</speaker>
 <p>Ähhmm! ...</p>
 <pb n="248"></pb>
 </sp>, <sp who="#lehmann">
 <speaker>LEHMANN.</speaker>
 <p>Ich weeß gor nich, wozu 's so vill Verrickte uff der Wält gibbt. Die sein doch zu gor
             nischt gutt! Woas? Su ä Schwindsichtiger, doas luß ich mer noch gefalln – ober su ä
             Verrickter ... doas is do gor kee Mensch nich! ... Chchfft ...</p>
 </sp>, <sp who="#krawutschke">
 <speaker>KRAWUTSCHKE.</speaker>
 <p>I nu heeren Se mal, nu sein Se so gut, nu halten Se oaber gleich de Luft on, joa?
             Ähhmm!</p>
 </sp>, <sp who="#lehmann">
 <speaker>LEHMANN.</speaker>
 <p>Nu – wozu sein se denn gutt, Härr Dukter ... de Verrickten ... Chchfft! ... woas?
             ...</p>
 <

In [78]:
i = 0
while i < len(sp):
    if type(sp[i].speaker) is not type(None): 
        if i+1 < len(sp):
            if type(sp[i+1].speaker) is not type(None): 
                if type(sp[i+1].p) is not type(None):
                    speaker1 = sp[i].speaker.text
                    speaker1 = re.sub(r"[^a-z]+", '', speaker1.lower())
                    speaker2 = sp[i+1].speaker.text
                    speaker2 = re.sub(r"[^a-z]+", '', speaker2.lower())
                    dict_name = "{}_{}".format(speaker2,speaker1)
                    text = sp[i+1].p.text
                    if dict_name in replik:
                        replik[dict_name][text] = 0
    i += 1

In [82]:
pprint(replik)

{'abramsen_angler': {},
 'abramsen_bulle': {"He he! Wat gieft 't denn? Wie heißt? Wat hewwt ju denn? Seggt mi doch ...; ick bin jo\n            jüwer Hauptmann. Seggt mol, ju hewwt mi doch zu jüwerm Hauptmann maken!": 0},
 'abramsen_helene': {},
 'abramsen_henriette': {},
 'abramsen_krawutschke': {},
 'abramsen_lehmann': {"He hat den ganzen Buddel utsupen! He het mi nischt laten! Er weint wie ein\n              Kind. Giff mi Snaps; giff mi 'n eenzigen Droppen Snaps! Er kauert sich\n              vor Lehmann nieder wie ein Hund, der schön macht, und bewegt die Hände wie\n              Pfoten. Wau, wau, giff mi Snaps!": 0,
                      "So'n Massel, ik hew Snaps! Ik hew Snaps un ju nich! ...": 0},
 'abramsen_lude': {'Heißt ä mießer Szoff, was nehmen werd der Butje! Wat will wi denn drinken? Bär? Snaps?\n            Wien? Seggt, ick wert all besorgen. Mir is all eins. Ick hewwt all da drinn –\n              Zeigt auf seinen Bauch. – seggt man blot, wat ick rutlaten sull ...': 0},

## Sentiment Analyse
### TextBlob

#### Gesagtem wird Sentiment zugeordnet

In [80]:
from pprint import pprint

In [60]:
for key in replik:
    for innerkey in replik[key]:
        senti = TB(innerkey)
        replik[key][innerkey] = senti.sentiment.polarity

In [58]:
replik

{'miller_millerin': {},
 'millerin_miller': {'Was gibst denn? Was ists denn?': 0.0},
 'miller_wurm': {'Dem Herrn einen Sessel, Frau. Wollens ablegen, Herr Landsmann?': 0.0,
  'Willst du dein Maul halten? Willst das Violonzello am Hirnkasten wissen? – Was kannst\n              du wissen? Was kann er gesagt haben? – Kehren sich an das Geklatsch nicht, Herr Vetter\n              – Marsch du in deine Küche – Werden mich doch nicht für des Dummkopfs leiblichen\n              Schwager halten, daß ich obenaus woll mit dem Mädel? Werden doch das nicht von mir\n              denken, Herr Sekretarius?': 0.0,
  'Willst du Arm und Bein entzwei haben, Wettermaul?': -1.0,
  "Daß dich alle Hagel! 's Mädel muß Sie kennen. Was ich alter Knasterbart an Ihnen\n              abgucke, ist just kein Fressen fürs junge naschhafte Mädel. Ich will Ihnen aufs Haar\n              hin sagen, ob Sie ein Mann fürs Orchester sind – aber eine Weiberseel ist auch für\n              einen Kapellmeister zu spitzig. – Un

#### Durchschnitt aller Sentiments eines Charakters wird gebildet und in neues Dict geschrieben

In [62]:
all_in_all = {}
for key in replik:
    x = 0
    for innerkey, value in replik[key].items():
        x = (x + value)
        all_in_all[key] = x
for key,value in all_in_all.items():
    all_in_all[key] = value/len(replik[key])

In [63]:
all_in_all

{'millerin_miller': 0.0,
 'miller_wurm': -0.21071428571428572,
 'wurm_miller': -0.05,
 'miller_luise': -0.031956845238095236,
 'luise_miller': 0.02863287250384025,
 'miller_ferdinand': 0.024714627477785366,
 'ferdinand_miller': 0.14527777777777776,
 'luise_millerin': 0.0,
 'wurm_luise': 0.025524781341107868,
 'luise_wurm': -0.021295756397797214,
 'ferdinand_wurm': -0.5,
 'luise_ferdinand': -0.08341545709192769,
 'ferdinand_luise': 0.07773832134943245,
 'luise_lady': 0.05648310023310023,
 'lady_luise': 0.05772413860649154,
 'ferdinand_lady': -0.001004166666666675,
 'lady_ferdinand': 0.0029857142857142873,
 'ferdinand_hofmarschall': 0.07007812499999999,
 'hofmarschall_ferdinand': 0.12395833333333334,
 'hofmarschall_lady': 0.05555555555555555,
 'lady_hofmarschall': -0.09628205128205126,
 'sophie_hofmarschall': -0.041666666666666664,
 'sophie_lady': 0.051470588235294115,
 'lady_sophie': -0.08479166666666668,
 'sophie_bedienter': 0.25,
 'bedienter_sophie': 0.0,
 'lady_bedienter': -0.5,
 'be